In [130]:
import pickle
import pandas as pd
# file_name = "D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\chennai_cars.xlsx"
df_bengalore =  pd.read_excel("D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\bangalore_cars.xlsx", sheet_name="bangalore_cars.csv")
df_chennai =  pd.read_excel("D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\chennai_cars.xlsx", sheet_name="chennai_cars.csv")
df_delhi =  pd.read_excel("D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\delhi_cars.xlsx", sheet_name="delhi_cars.csv")
df_hyderabad =  pd.read_excel("D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\hyderabad_cars.xlsx", sheet_name="hyderabad_cars.csv")
df_jaipur =  pd.read_excel("D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\jaipur_cars.xlsx", sheet_name="jaipur_cars.csv")
df_kolkata =  pd.read_excel("D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\kolkata_cars.xlsx", sheet_name="kolkata_cars.csv")

# all_dt = [df_bengalore, df_chennai, df_delhi, df_hyderabad, df_jaipur, df_kolkata]

# all_dt
#info = str(info)
# with open("D:\\Bhuvii\\GUVI\\data\\user_info.pkl", mode="wb+") as my_file:
    # pickle.dump(info, my_file)

In [161]:
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

file_names = ['bangalore_cars', 'chennai_cars', 'delhi_cars', 'hyderabad_cars', 'jaipur_cars', 'kolkata_cars']
dfs = pd.DataFrame()

# Function to add quotes around unquoted values
def add_quotes(json_str):
    json_str = re.sub(r'(\w+):', r'"\1":', json_str)  # Add quotes around keys
    json_str = re.sub(r': (\w+)(,|})', r': "\1"\2', json_str)  # Add quotes around values
    return json_str

# Convert Price Column
def convert_price(price_str):
    price_str = price_str.replace(",", "").strip()
    if float(price_str) < 99 :
        return float(price_str) * 100000
    return float(price_str)

df = pd.concat(pd.read_excel(r'D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\'+ file +'.xlsx', sheet_name = file + ".csv")
               for file in file_names)

df_lst = []

for file in file_names :
    # Load the dataset
    df = pd.read_excel(r'D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\'+ file +'.xlsx', sheet_name = file + ".csv")
    
    # Apply the function to the JSON column
    json_column = 'new_car_detail'  # Replace with your actual column name
    drop_columns = ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs', 'car_links']
    df[json_column] = df[json_column].apply(add_quotes)

    # Convert single quotes to double quotes in the JSON column
    df[json_column] = df[json_column].apply(lambda x: x.replace("'", '"').replace("\"https\"", "https").replace('₹', '').replace('Lakh', '').replace('Crore', ''))

    # Parse JSON data from the specific column
    df[json_column] = df[json_column].apply(json.loads)

    # print(df[json_column][0])

    # Expand JSON data into separate columns
    json_df = pd.json_normalize(df[json_column])
    df = df.drop(columns=drop_columns).join(json_df)

    # Convert the price column
    df['price'] = df['price'].apply(convert_price)
    # Data Preprocessing
    df = df.dropna()  # Drop missing values
    df = df.drop(columns=['it', 'ownerNo', 'owner', 'oem', 'model', 'modelYear', 'centralVariantId', 'variantName', 'priceActual', 'priceSaving', 'priceFixedText', 'trendingText.imgUrl','trendingText.heading', 'trendingText.desc'])
    
    df["city"] = file.replace("_cars", "")
    # print(file)
    df_lst.append(df)

overall_df = pd.concat(df_lst, ignore_index=True)

# Convert 'km' column to numerical values
overall_df['km'] = overall_df['km'].str.replace(',', '').astype(float)
overall_df = pd.get_dummies(overall_df, drop_first=True)  # Encode categorical variables
# overall_df

# Feature Selection
X = overall_df.drop('price', axis=1)  # Features
y = overall_df['price']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

with open(r'D:\\Bhuvii\\GUVI\\Projects\\4\\Car-Price-Prediction-Project-main\\Dataset\\car_price_model.pkl', 'wb') as file:
    pickle.dump(model, file)
    
# Prediction
new_data = pd.DataFrame({
    'ft_Lpg': ['Lpg'],
    'bt_SUV': ['SUV'],
    'km': [50000],
    'transmission_Manual': ['Manual'],
    'city_chennai': ['chennai']
})
new_data = pd.get_dummies(new_data, drop_first=True)

# Ensure new_data has the same columns as X_train
missing_cols = set(X_train.columns) - set(new_data.columns)
for col in missing_cols:
    new_data[col] = 0
new_data = new_data[X_train.columns]

# Print new_data to verify its content
print(new_data.columns)

predicted_price = model.predict(new_data)
print(f'Predicted Price: {predicted_price[0]}')

Mean Absolute Error: 494515.2531017461
Index(['km', 'ft_Diesel', 'ft_Electric', 'ft_Lpg', 'ft_Petrol',
       'bt_Convertibles', 'bt_Coupe', 'bt_Hatchback', 'bt_Hybrids', 'bt_MUV',
       'bt_Minivans', 'bt_Pickup Trucks', 'bt_SUV', 'bt_Sedan', 'bt_Wagon',
       'transmission_Manual', 'city_chennai', 'city_delhi', 'city_hyderabad',
       'city_jaipur', 'city_kolkata'],
      dtype='object')
Predicted Price: 1122283.609069423
